## Indexing in Databases

Database Index, as the name suggests, maintains an index to the actual data and thereby improves performance to retrieve data from the actual table. In a more database terminology, the index allows fetching page containing indexed data in a very minimal traversal as data is sorted in specific order. Index benefit comes at the cost of additional storage space in order to write additional data. Indexes are specific to the underlying table and consist of one or more keys (i.e. one or more columns of the specified table). There are primarily two types of index architecture

- **Clustered Index** – Index data gets stored along with other part of data and data gets sorted based on index key. At most there can be only one index in this category for a specified table.
- **Non-Clustered Index** – Index data gets stored separately and it has a pointer to the storage where other part of data is stored. This is also known as secondary index. There can be as many indexes of this category as you want on a specified table.

There are various data structures used for implementing indexes, some of the widely adopted by the majority of databases are B-Tree and Hash.

https://timilearning.com/posts/data-storage-on-disk/part-one/


## Hash Indexes

A simple key value pair index. Let’s say our data storage consists only of appending to a file, as in the preceding example. Then the simplest possible indexing strategy is this: keep an in-memory
hash map where every key is mapped to a byte offset in the data file—the location at
which the value can be found. Whenever you append a
new key-value pair to the file, you also update the hash map to reflect the offset of the
data you just wrote (this works both for inserting new keys and for updating existing
keys). When you want to look up a value, use the hash map to find the offset in the
data file, seek to that location, and read the value.


**Tombstone:** If you want to delete a key and its associated value, you have to append a special
deletion record to the data file (sometimes called a tombstone). When log seg‐
ments are merged, the tombstone tells the merging process to discard any previ‐
ous values for the deleted key.

**Crash Recovery:**
If the database is restarted, the in-memory hash maps are lost. In principle, you
can restore each segment’s hash map by reading the entire segment file from
beginning to end and noting the offset of the most recent value for every key as
you go along. However, that might take a long time if the segment files are large,
which would make server restarts painful. To avoid this we can snapshot of each segment’s hash map on disk, which can be loaded into memory more quickly.

### However, the hash table index also has limitations:
• The hash table must fit in memory, so if you have a very large number of keys,
you’re out of luck. In principle, you could maintain a hash map on disk, but
unfortunately it is difficult to make an on-disk hash map perform well. It
requires a lot of random access I/O, it is expensive to grow when it becomes full,
and hash collisions require fiddly logic.

• Range queries are not efficient. For example, you cannot easily scan over all keys
between kitty00000 and kitty99999 you’d have to look up each key individually in the hash maps.

### In-Memory index (KV Pairs)

In [2]:
import os 
def encode(value):
    return value.encode('unicode-escape').decode('ASCII')

def decode(value):
    return value.encode("ASCII").decode('unicode-escape')

class HashIndex:
    def __init__(self, name):
        self.name = f"data/{name}.kv" # Filename
        if not os.path.exists(self.name):
            with open(self.name, 'w'):
                pass
        self.index = {}
    
    # Special characters <EOK> end of key & <EOP> end of pair

    def insert(self, key, value):
        # Write to file and save offset
        with open(self.name, 'a') as f:
            f.seek(0, 2) # first line from the end
            end_offset = f.tell() # end offset
            f.write(encode(f"{key}<EOK>{value}<EOP>"))
        # create index using offset
        self.index[key] = {
            "offset": end_offset,
            "pk_size": len(f"{key}"), 
            "data_size": len(f"{value}"),
            "data": f"{value}"
        }
        
    def get(self, key):
        # Read from Offset and length
        try:
            return self.index[key]["data"]
        except:
            return "<NAN>"

    def iterate(self, keys):
        return list(map(lambda x : self.get(x), keys))

    def delete(self,key):
        key_index = self.index[key]
        with open(self.name, 'a') as f:
            char = f.seek(key_index["offset"])
            # Tombstone
            f.write(encode(f"{key}<EOK><NAN><EOP>"))
        self.index.pop(key, None)

In [3]:
infile = HashIndex("hash_test")
infile.insert(1, {"name":"bheem"}) # Dict
infile.insert(2, '😎') # Emoji
infile.insert(3, "Some Stupid String") # String
infile.insert(4, 120293292939393) # Big Number
print(infile.index)

{1: {'offset': 251, 'pk_size': 1, 'data_size': 17, 'data': "{'name': 'bheem'}"}, 2: {'offset': 279, 'pk_size': 1, 'data_size': 1, 'data': '😎'}, 3: {'offset': 300, 'pk_size': 1, 'data_size': 18, 'data': 'Some Stupid String'}, 4: {'offset': 329, 'pk_size': 1, 'data_size': 15, 'data': '120293292939393'}}


### Operations

In [8]:
# Read
print(infile.iterate([1,2,3,4,5,6]))
print(infile.get(2))

["{'name': 'bheem'}", '😎', 'Some Stupid String', '120293292939393', '<NAN>', '<NAN>']
😎


In [4]:
# Create
assert infile.get(1) == "{'name': 'bheem'}"
assert infile.get(3) == 'Some Stupid String'
assert infile.get(4) == '120293292939393'
infile.iterate([3, 1]) == ['Some Stupid String', "{'name': 'bheem'}"]
# Update
infile.insert(4, 'update 4th index') # Big Number
assert infile.get(4) == 'update 4th index'
# Delete
infile.delete(4)
assert infile.get(4) == '<NAN>'

### Read from disk

When database restarts the index needs to be re-read back into memory

In [5]:
def deserialize(filename):
    with open(f'data/{filename}.kv') as f:
        lines = f.readline()
    _index = {}
    offset_passed = 0
    for val in lines.split("<EOP>")[:-1]:
        key, value = val.split("<EOK>")
        _index[int(key)] = {
            "offset": offset_passed,
            "pk_size": len(key), 
            "data_size": len(value),
            "data": decode(f"{value}")
        }
        offset_passed += len(val) + 5
    hashin = HashIndex("hash_test")
    hashin.index = _index
    return hashin

In [6]:
infile = deserialize("hash_test")
infile.iterate([1,2,3,4])

["{'name': 'bheem'}", '😎', 'Some Stupid String', '<NAN>']

## Tree indexes

We can make a simple change to the format of our segment files: we require that the sequence of key-value pairs is sorted by key

Maintaining a sorted structure in memory is super easy and There are plenty of well-known tree data
structures that you can use, such as red-black trees or AVL trees. With these data
structures, you can insert keys in any order and read them back in sorted order. Also with these Insertion Deletion and Search complexity are O(log n) so they don't increase exponentially or by N factor


**LSM Trees**


Here is how LSM Indexes work with Balanced trees like AVL, Red Black 


-  When a write comes in, add it to an in-memory balanced tree data structure (for
example, a red-black tree). This in-memory tree is sometimes called a memtable.
- When the memtable gets bigger than some threshold—typically a few megabytes
—write it out to disk as an SSTable file. This can be done efficiently because the
tree already maintains the key-value pairs sorted by key. The new SSTable file
becomes the most recent segment of the database. While the SSTable is being
written out to disk, writes can continue to a new memtable instance.
- In order to serve a read request, first try to find the key in the memtable, then in
the most recent on-disk segment, then in the next-older segment, etc.
- From time to time, run a merging and compaction process in the background to
combine segment files and to discard overwritten or deleted values.


AVL Trees
: https://www.programiz.com/dsa/avl-tree

Red-Black Trees
: https://www.youtube.com/playlist?list=PL9xmBV_5YoZNqDI8qfOZgzbqahCUmUEin


**BTree**

We discussed AVL trees for LSM indexes above but we can also resort to Btree which is a more commonly used index

Balanced Trees like binary search tree, avl tree and red-black tree can store only one key in one node. If you have to store a large number of keys, then the height of such trees becomes very large and the access time increases. However, B-tree can store many keys in a single node and can have multiple child nodes. This decreases the height significantly allowing faster disk accesses.

B tree: https://www.programiz.com/dsa/b-tree

The log-structured indexes we saw earlier break the database down into variable-size
segments, typically several megabytes or more in size, and always write a segment
sequentially. By contrast, B-trees break the database down into fixed-size blocks or
pages, traditionally 4 KB in size (sometimes bigger), and read or write one page at a
time. This design corresponds more closely to the underlying hardware, as disks are
also arranged in fixed-size blocks.

LSM-trees are typically faster for writes, whereas B-trees
are thought to be faster for reads. Reads are typically slower on LSM-trees
because they have to check several different data structures and SSTables at different
stages of compaction.

Gotta love Indians on the internet :) https://www.youtube.com/watch?v=aZjYr87r1b8 An excellent Btree resource

AVL trees are intended for in-memory use, where random access is relatively cheap. B-trees are better suited for disk-backed storage, because they group a larger number of keys into each node to minimize the number of seeks required by a read or write operation. (This is why B-trees are often used in file systems and databases, such as SQLite.)

**B+Trees:** https://www.programiz.com/dsa/b-plus-tree

https://github.com/NicolasLM/bplustree

https://stackoverflow.com/questions/2734692/avl-tree-vs-b-tree/2734720

In [2]:
class Node(object):
    def __init__(self, key, data=None):
        self.key = key
        self.data = str(data)
        self.left = None
        self.right = None
        self.height = 1

### AVL Tree

In [1]:
class AvlTreeIndex(object):

    # Search through the AvlTreeIndex
    def get(self, root, val):
        if root is None:
            return None
        elif (root.key == val):
            return root
        elif(root.key < val):
            return self.get(root.right,val)
        return self.get(root.left,val)

    # Function to insert a node
    def insert(self, root, key, data):
        node = self.get(root, key)
        # Find the correct location and insert the node
        if not root:
            return Node(key=key, data=data)
        # Node already exists
        elif node:
            node.data = data
        elif key < root.key:
            root.left = self.insert(root.left, key, data)
        else:
            root.right = self.insert(root.right, key, data)

        root.height = 1 + max(self.get_height(root.left),
                              self.get_height(root.right))

        # Update the balance factor and balance the tree
        balanceFactor = self.get_balance(root)
        if balanceFactor > 1:
            if key < root.left.key:
                return self.rotate_right(root)
            else:
                root.left = self.rotate_left(root.left)
                return self.rotate_right(root)

        if balanceFactor < -1:
            if key > root.right.key:
                return self.rotate_left(root)
            else:
                root.right = self.rotate_right(root.right)
                return self.rotate_left(root)

        return root

    # Function to delete a node
    def delete(self, root, key):

        # Find the node to be deleted and remove it
        if not root:
            return root
        elif key < root.key:
            root.left = self.delete(root.left, key)
        elif key > root.key:
            root.right = self.delete(root.right, key)
        else:
            if root.left is None:
                temp = root.right
                root = None
                return temp
            elif root.right is None:
                temp = root.left
                root = None
                return temp
            temp = self.min_node(root.right)
            root.key = temp.key
            root.right = self.delete(root.right,
                                          temp.key)
        if root is None:
            return root

        # Update the balance factor of nodes
        root.height = 1 + max(self.get_height(root.left),
                              self.get_height(root.right))

        balanceFactor = self.get_balance(root)

        # Balance the tree
        if balanceFactor > 1:
            if self.get_balance(root.left) >= 0:
                return self.rotate_right(root)
            else:
                root.left = self.rotate_left(root.left)
                return self.rotate_right(root)
        if balanceFactor < -1:
            if self.get_balance(root.right) <= 0:
                return self.rotate_left(root)
            else:
                root.right = self.rotate_right(root.right)
                return self.rotate_left(root)
        return root

    # Function to perform left rotation
    def rotate_left(self, z):
        y = z.right
        T2 = y.left
        y.left = z
        z.right = T2
        z.height = 1 + max(self.get_height(z.left),
                           self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left),
                           self.get_height(y.right))
        return y

    # Function to perform right rotation
    def rotate_right(self, z):
        y = z.left
        T3 = y.right
        y.right = z
        z.left = T3
        z.height = 1 + max(self.get_height(z.left),
                           self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left),
                           self.get_height(y.right))
        return y

    # Get the height of the node
    def get_height(self, root):
        if not root:
            return 0
        return root.height

    # Get balance factor of the node
    def get_balance(self, root):
        if not root:
            return 0
        return self.get_height(root.left) - self.get_height(root.right)

    def min_node(self, root):
        if root is None or root.left is None:
            return root
        return self.min_node(root.left)

    def show(self, root):
        if root:
            print(root.key)
            self.show(root.left)
            self.show(root.right)

### Operations

In [3]:
tindex = AvlTreeIndex()
# Create
nums = [(1, {"a": "apple"}), (2, 423), (3, "Some String")]
root = None
for num in nums:
    root = tindex.insert(root, num[0], num[1])
    
# Read
print(tindex.get(root, 1).data)

# Update
root = tindex.insert(root, 1, "Updated Value")
print(tindex.get(root, 1).data)

# Delete 
root = tindex.delete(root, 1)
print(tindex.get(root, 1))

{'a': 'apple'}
Updated Value
None


### Btree

In [4]:
# Btree node
class BTreeNode:
    def __init__(self, leaf=False):
        self.leaf = leaf
        self.keys = []
        self.child = []


class BTree:
    def __init__(self, t):
        self.root = BTreeNode(True)
        self.t = t

    # Print the tree
    def print_tree(self, x):
        for i in x.keys:
            print(i)
        if len(x.child) > 0:
            for i in x.child:
                self.print_tree(i)

    # Search key
    def search_key(self, k, x=None):
        if x is not None:
            i = 0
            while i < len(x.keys) and k > x.keys[i][0]:
                i += 1
            if i < len(x.keys) and k == x.keys[i][0]:
                return (x, i)
            elif x.leaf:
                return None
            else:
                return self.search_key(k, x.child[i])
        else:
            return self.search_key(k, self.root)
    
    # Insert a key
    def insert(self, k):
        root = self.root
        existing = self.search_key(k[0])
        if existing:
            _existing, _index = existing
            return _existing.keys[_index][1].append(k[1])
        if len(root.keys) == (2 * self.t) - 1:
            temp = BTreeNode()
            self.root = temp
            temp.child.insert(0, root)
            self.split_child(temp, 0)
            self.insert_non_full(temp, k)
        else:
            self.insert_non_full(root, k)

    # Insert non full
    def insert_non_full(self, x, k):
        i = len(x.keys) - 1
        if x.leaf:
            x.keys.append((None, None))
            while i >= 0 and k[0] < x.keys[i][0]:
                x.keys[i + 1] = x.keys[i]
                i -= 1
            x.keys[i + 1] = k
        else:
            while i >= 0 and k[0] < x.keys[i][0]:
                i -= 1
            i += 1
            if len(x.child[i].keys) == (2 * self.t) - 1:
                self.split_child(x, i)
                if k[0] > x.keys[i][0]:
                    i += 1
            self.insert_non_full(x.child[i], k)

    # Split the child
    def split_child(self, x, i):
        t = self.t
        y = x.child[i]
        z = BTreeNode(y.leaf)
        x.child.insert(i + 1, z)
        x.keys.insert(i, y.keys[t - 1])
        z.keys = y.keys[t: (2 * t) - 1]
        y.keys = y.keys[0: t - 1]
        if not y.leaf:
            z.child = y.child[t: 2 * t]
            y.child = y.child[0: t - 1]

    # Delete a node
    def delete(self, x, k):
        t = self.t
        i = 0
        while i < len(x.keys) and k[0] > x.keys[i][0]:
            i += 1
        if x.leaf:
            if i < len(x.keys) and x.keys[i][0] == k[0]:
                x.keys.pop(i)
                return
            return

        if i < len(x.keys) and x.keys[i][0] == k[0]:
            return self.delete_internal_node(x, k, i)
        elif len(x.child[i].keys) >= t:
            self.delete(x.child[i], k)
        else:
            if i != 0 and i + 2 < len(x.child):
                if len(x.child[i - 1].keys) >= t:
                    self.delete_sibling(x, i, i - 1)
                elif len(x.child[i + 1].keys) >= t:
                    self.delete_sibling(x, i, i + 1)
                else:
                    self.delete_merge(x, i, i + 1)
            elif i == 0:
                if len(x.child[i + 1].keys) >= t:
                    self.delete_sibling(x, i, i + 1)
                else:
                    self.delete_merge(x, i, i + 1)
            elif i + 1 == len(x.child):
                if len(x.child[i - 1].keys) >= t:
                    self.delete_sibling(x, i, i - 1)
                else:
                    self.delete_merge(x, i, i - 1)
            self.delete(x.child[i], k)

    # Delete internal node
    def delete_internal_node(self, x, k, i):
        t = self.t
        if x.leaf:
            if x.keys[i][0] == k[0]:
                x.keys.pop(i)
                return
            return

        if len(x.child[i].keys) >= t:
            x.keys[i] = self.delete_predecessor(x.child[i])
            return
        elif len(x.child[i + 1].keys) >= t:
            x.keys[i] = self.delete_successor(x.child[i + 1])
            return
        else:
            self.delete_merge(x, i, i + 1)
            self.delete_internal_node(x.child[i], k, self.t - 1)

    # Delete the predecessor
    def delete_predecessor(self, x):
        if x.leaf:
            return x.pop()
        n = len(x.keys) - 1
        if len(x.child[n].keys) >= self.t:
            self.delete_sibling(x, n + 1, n)
        else:
            self.delete_merge(x, n, n + 1)
        self.delete_predecessor(x.child[n])

    # Delete the successor
    def delete_successor(self, x):
        if x.leaf:
            return x.keys.pop(0)
        if len(x.child[1].keys) >= self.t:
            self.delete_sibling(x, 0, 1)
        else:
            self.delete_merge(x, 0, 1)
        self.delete_successor(x.child[0])

    # Delete resolution
    def delete_merge(self, x, i, j):
        cnode = x.child[i]

        if j > i:
            rsnode = x.child[j]
            cnode.keys.append(x.keys[i])
            for k in range(len(rsnode.keys)):
                cnode.keys.append(rsnode.keys[k])
                if len(rsnode.child) > 0:
                    cnode.child.append(rsnode.child[k])
            if len(rsnode.child) > 0:
                cnode.child.append(rsnode.child.pop())
            new = cnode
            x.keys.pop(i)
            x.child.pop(j)
        else:
            lsnode = x.child[j]
            lsnode.keys.append(x.keys[j])
            for i in range(len(cnode.keys)):
                lsnode.keys.append(cnode.keys[i])
                if len(lsnode.child) > 0:
                    lsnode.child.append(cnode.child[i])
            if len(lsnode.child) > 0:
                lsnode.child.append(cnode.child.pop())
            new = lsnode
            x.keys.pop(j)
            x.child.pop(i)

        if x == self.root and len(x.keys) == 0:
            self.root = new

    # Delete the sibling
    def delete_sibling(self, x, i, j):
        cnode = x.child[i]
        if i < j:
            rsnode = x.child[j]
            cnode.keys.append(x.keys[i])
            x.keys[i] = rsnode.keys[0]
            if len(rsnode.child) > 0:
                cnode.child.append(rsnode.child[0])
                rsnode.child.pop(0)
            rsnode.keys.pop(0)
        else:
            lsnode = x.child[j]
            cnode.keys.insert(0, x.keys[i - 1])
            x.keys[i - 1] = lsnode.keys.pop()
            if len(lsnode.child) > 0:
                cnode.child.insert(0, lsnode.child.pop())

### Operations

In [5]:
# CREATE
B = BTree(3)
for i in range(10):
    B.insert((i,[i+2]))

# UPDATE
B.insert((3, 7))
B.insert((0, 7))

B.print_tree(B.root)

# DELETE
B.delete(B.root, (0,))
print("\n")
B.print_tree(B.root)

(2, [4])
(5, [7])
(0, [2, 7])
(1, [3])
(3, [5, 7])
(4, [6])
(6, [8])
(7, [9])
(8, [10])
(9, [11])


(5, [7])
(1, [3])
(2, [4])
(3, [5, 7])
(4, [6])
(6, [8])
(7, [9])
(8, [10])
(9, [11])


In [6]:
print(vars(B.root)), print(len(B.root.child))
print("\n")
print(vars(B.root.child[0]))
print("\n")
print(vars(B.root.child[1]))

{'leaf': False, 'keys': [(5, [7])], 'child': [<__main__.BTreeNode object at 0x7fb970b73208>, <__main__.BTreeNode object at 0x7fb970b73518>]}
2


{'leaf': True, 'keys': [(1, [3]), (2, [4]), (3, [5, 7]), (4, [6])], 'child': []}


{'leaf': True, 'keys': [(6, [8]), (7, [9]), (8, [10]), (9, [11])], 'child': []}


In [7]:
print("""3
[(5, [7])] 2
[(1, [3]), (2, [4]), (3, [5, 7]), (4, [6])] 0 
[(6, [8]), (7, [9]), (8, [10]), (9, [11])] 0
"""
)
btree_serialized = "3<TREE>[(5, [7])]:2<TREE>[(1, [3]),(2, [4]),(3,[5, 7]),(4, [6])]:0<TREE>[(6, [8]),(7, [9]),(8, [10]),(9, [11])]:0<TREE>"

3
[(5, [7])] 2
[(1, [3]), (2, [4]), (3, [5, 7]), (4, [6])] 0 
[(6, [8]), (7, [9]), (8, [10]), (9, [11])] 0



In [9]:
import ast
import itertools
bts = btree_serialized.split("<TREE>")
t, serialized = bts[0], bts[1:]
s_tokens = itertools.chain(*[s.split(':') for s in serialized])

B = BTree(t)
for s_token in s_tokens:
    try:
        t_val, count = s_token, int(next(s_tokens))
        if count != 0:
            node = B.root
            node.leaf=False
            node.keys = ast.literal_eval(t_val)
            for i in range(count):
                t_val, count = next(s_tokens), int(next(s_tokens))
                B.root.child[1]
    except StopIteration:
        pass

Other Indexes:

- Multidimensional Indexes (For GIS data) i,e Latitude and Longitude
- Full text search & Fuzzy Indexes

### Material 

- https://cstack.github.io/db_tutorial/
- https://tikv.github.io/deep-dive-tikv/overview/introduction.html